In [196]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
from bs4.element import Comment, NavigableString

from random import randint
from time import sleep

In [8]:
# загружаем страницу со всеми названиями альбомов и песен

In [118]:
def load_page(name, mode):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}
    
    if mode=='artist':
        url = f'https://www.azlyrics.com/{name[0]}/{name}.html'
        print(url)
        r = requests.get(url, headers = headers)
        with open(name+'.html', 'w') as output_file:
            output_file.write(r.text)
            
    elif  mode=='song':
        url = f'https://www.azlyrics.com/{name[3:]}'
        r = requests.get(url, headers = headers)
        with open('song.html', 'w') as output_file:
            output_file.write(r.text)

In [104]:
def read_file(filename):
    with open(filename) as input_file:
        text = input_file.read()
    return text


def parse_user_datafile_bs(filename):
    results = []
    text = read_file(filename)
    soup = BeautifulSoup(text, 'lxml')
    song_list = soup.find('div', {'id': 'listAlbum'})
    song_list = song_list.find_all('a')[1:]

    for song in song_list:
        if 'href' in str(song):
            results.append({
                'song_name':song.contents[0] ,
                'lyrics_link': song.get('href')
            })
        
    return results


# Identify the start comment
def isInstanceBeginEditable(text):
    return (isinstance(text, Comment) and
            text.strip().startswith("Usage of azlyrics.com content by any third-party lyrics provider is prohibited by our licensing agreement. Sorry about that."))

# Identify the end comment
def isInstanceEndEditable(text):
    return (isinstance(text, Comment) and
            text.strip().startswith("MxM banner"))

In [105]:
# load_page('linkinpark', mode='artist')
res = parse_user_datafile_bs('linkinpark.htm')

'BeautifuL'

In [197]:
df = pd.DataFrame(columns=["Year","Album","Song", "Text"])

for song in results:
    
    sleep(randint(10,30))
    
    load_page(song['lyrics_link'], mode='song')

    text = read_file('song.html')
    soup = BeautifulSoup(text, 'lxml')

    song_text = []
    # Look for start comments
    for instanceBeginEditable in soup.find_all(text=isInstanceBeginEditable):
        # We found a start comment, look at all text and comments:
        for text in instanceBeginEditable.find_all_next(text=True):
            # We found a text or comment, examine it closely
            if isInstanceEndEditable(text):
                # We found the end comment, everybody out of the pool
                break
            if isinstance(text, Comment):
                # We found a comment, ignore
                continue
            if not text.strip():
                # We found a blank text, ignore
                continue
            # Whatever is left must be good
            song_text.append(text[1:])
            #removes duplicated lines
            song_text = [ii for n,ii in enumerate(song_text) if ii not in song_text[:n]]


    df = df.append({
         "Year": soup.find('div', {'class': 'panel album-panel noprint'}).find('a').contents[0].split('(', 1)[1].split(')')[0],
         "Album":  soup.find('div', {'class': 'panel album-panel noprint'}).find('a').contents[0].split('"', 1)[1].split('"')[0],
         "Song": song['song_name'],
         "Text": song_text
          }, ignore_index=True)


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response',))

In [194]:
df.to_csv('all_albums.csv', index=False)